In [1]:
import os
import binascii
import ecdsa
import hmac
import hashlib

In [2]:
seed = os.urandom(32)
root_key = b"Bitcoin seed"

In [7]:
def hmac_sha512(data, keymessage):
    hash = hmac.new(data, keymessage, hashlib.sha512).digest()
    return hash

In [12]:
def create_pubkey(private_key):
    publickey = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1).verifying_key.to_string()
    return publickey

In [13]:
master = hmac_sha512(seed, root_key)
master_secretkey = master[:32]
master_chaincode = master[32:]
master_publickey = create_pubkey(master_secretkey)

In [14]:
master_publickey_integer = int.from_bytes(master_publickey[32:], byteorder='big')

In [15]:
if master_publickey_integer % 2 == 0:
    master_publickey_x = b"\x02" + master_publickey[:32]
else:
    master_publickey_x = b"\x03" + master_publickey[:32]

In [16]:
# マスター秘密鍵
print('マスター秘密鍵')
print(binascii.hexlify(master_secretkey))
print('\n')
# マスターチェーンコード
print('マスターチェーンコード')
print(binascii.hexlify(master_chaincode))
print('\n')
# マスター圧縮公開鍵
print('マスター公開鍵')
print(binascii.hexlify(master_publickey_x))

マスター秘密鍵
b'ee1192474adfa8c415cd7b29d3f47fe15478289c8b5105c444cd0f7f4214fa9b'


マスターチェーンコード
b'288a7f66ed45910e1b4c9a71a68ac67f1d17e075a76f8dfffda7264a3b906393'


マスター公開鍵
b'028f275b58e6cdd70d1cb30f0434abd98e1da4c79ae4c7f95c60b574a237fb2440'


In [17]:
index = 0
index_bytes = index.to_bytes(8, "big")
data = master_publickey_x + index_bytes
result_hmac512 = hmac_sha512(data, master_chaincode)

In [18]:
sum_integer = int.from_bytes(master_secretkey, "big") + int.from_bytes(result_hmac512[:32], "big")

In [19]:
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1
child_secretkey = (sum_integer % p).to_bytes(32, "big")

In [21]:
# 子秘密鍵
print("子秘密鍵")
print(binascii.hexlify(child_secretkey))

子秘密鍵
b'a72ceb16d21f4f5c75240805d0339cf52e1164508a15b77a65560243c72797b7'
